# 1. Initial Steps and Data Setup

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import random_split

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
SEED = 42

np.random.seed(SEED)
torch.manual_seed(SEED)

In [ ]:
DATA_DIRECTORY = '/content/drive/MyDrive/Thesis/Data/hyperaktiv_with_controls/hyperaktiv_with_controls/'
VALID_IDs = [1, 3, 5, 7, 9, 11, 15, 19, 20, 21, 22, 23, 24, 27, 31, 32, 33, 34, 35, 36, 37, 39, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 55, 56, 57, 58, 59, 60, 61, 63, 64, 68, 71, 73, 75, 77, 78, 79, 81, 82, 83, 84, 85, 87, 88, 89, 90, 91, 93, 94, 95, 97, 98, 101, 104, 105]

In [ ]:
demographic_data = pd.read_csv(f'{DATA_DIRECTORY}patient_info.csv', sep=';')
#plot the balance of the ADHD class in the demographic_data for every record that has ID in VALID_IDs. Insert labels and make it more appealing
demographic_data = demographic_data[demographic_data['ID'].isin(VALID_IDs)]
# Extract labels for these IDs
labels = demographic_data['ADHD'].values

# Output the labels to verify
print(labels)

demographic_data['ADHD'].value_counts().plot(kind='bar', title='ADHD class balance in the dataset')
plt.xticks([0, 1], ['Control', 'ADHD'], rotation=0)
plt.ylabel('Number of records')
plt.show()

In [ ]:
# I want to noe the IDS of the control and ADHD patients
control_ids = demographic_data[demographic_data['ADHD'] == 0]['ID'].values
adhd_ids = demographic_data[demographic_data['ADHD'] == 1]['ID'].values

print(f'Number of control patients: {len(control_ids)}; IDS: {control_ids}')
print(f'Number of ADHD patients: {len(adhd_ids)}; IDS: {adhd_ids}')

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

def enhanced_split_dataset(ids, labels, train_ratio=0.80, val_ratio=0.10, test_ratio=0.10, random_seed=42):
    # Convert ratios to a useable format for StratifiedShuffleSplit
    splits = StratifiedShuffleSplit(n_splits=1, test_size=test_ratio, random_state=random_seed)

    # Remaining ratio is for validation
    remaining_ratio = 1.0 - test_ratio
    val_relative_ratio = val_ratio / remaining_ratio

    # First split to separate out the test set
    train_val_ids, test_ids = next(splits.split(ids, labels))

    # Second split to separate out the validation set from the remaining train set
    train_val_split = StratifiedShuffleSplit(n_splits=1, test_size=val_relative_ratio, random_state=random_seed)
    train_ids, val_ids = next(train_val_split.split(ids[train_val_ids], labels[train_val_ids]))

    # Convert indices to actual IDs
    train_ids = ids[train_ids]
    val_ids = ids[val_ids]
    test_ids = ids[test_ids]

    return train_ids, val_ids, test_ids

# Example usage:
# Assume labels is an array of labels corresponding to VALID_IDs in the same order
train_ids, val_ids, test_ids = enhanced_split_dataset(np.array(VALID_IDs), np.array(labels))


In [ ]:
demographic_data_train = demographic_data[demographic_data['ID'].isin(train_ids)]
demographic_data_train['ADHD'].value_counts().plot(kind='bar', title='ADHD class balance in the train dataset')
plt.xticks([0, 1], ['Control', 'ADHD'], rotation=0)
plt.ylabel('Number of records')
plt.show()

demographic_data_test = demographic_data[demographic_data['ID'].isin(test_ids)]
demographic_data_test['ADHD'].value_counts().plot(kind='bar', title='ADHD class balance in the test dataset')
plt.xticks([0, 1], ['Control', 'ADHD'], rotation=0)
plt.ylabel('Number of records')
plt.show()

demographic_data_val = demographic_data[demographic_data['ID'].isin(val_ids)]
demographic_data_val['ADHD'].value_counts().plot(kind='bar', title='ADHD class balance in the valdiation dataset')
plt.xticks([0, 1], ['Control', 'ADHD'], rotation=0)
plt.ylabel('Number of records')
plt.show()

In [ ]:

def load_data(sample, demographic_data):
    patients_data = {}  # Dictionary to store data

    for patient_id in sample:
        hrv_data = pd.read_csv(f'{DATA_DIRECTORY}/hrv_data/patient_hr_{patient_id}.csv', sep=';')
        activity_data = pd.read_csv(f'{DATA_DIRECTORY}/activity_data/patient_activity_{patient_id}.csv', sep=';')
        labels =  demographic_data[demographic_data['ID'] == patient_id]['ADHD'].values[0]  # Get the ADHD label for the patient

    # # Convert TIMESTAMP to datetime
    #     hrv_data['TIMESTAMP'] = pd.to_datetime(hrv_data['TIMESTAMP'], errors='coerce')
    #     activity_data['TIMESTAMP'] = pd.to_datetime(activity_data['TIMESTAMP'], errors='coerce')

    # Setting TIMESTAMP as index and checking for NaNs in data columns
        df_hrv = pd.DataFrame(data=hrv_data).set_index('TIMESTAMP')
        df_activity = pd.DataFrame(data=activity_data).set_index('TIMESTAMP')

    # # Fill NaNs in HRV and Activity before resampling
    #     df_hrv['HRV'] = df_hrv['HRV'].fillna(method='ffill')  # Forward fill as an example
    #     df_activity['ACTIVITY'] = df_activity['ACTIVITY'].fillna(method='ffill')  # Forward fill as an example

    # # Now resample
    #     df_hrv = df_hrv.resample('1T').mean()
    #     df_activity = df_activity.resample('1T').mean()

    # Trim datasets to the same length
        min_length = min(len(df_hrv), len(df_activity))
        df_hrv = df_hrv.iloc[:min_length]
        df_activity = df_activity.iloc[:min_length]

    # Store in dictionary
        patients_data[patient_id] = {
        'hrv': df_hrv,
        'activity': df_activity,
        'adhd': labels
        }

    return patients_data


train_data = load_data(train_ids, demographic_data=demographic_data_train)
test_data = load_data(test_ids, demographic_data=demographic_data_test)
val_data = load_data(val_ids, demographic_data=demographic_data_val)

all_data = {
    'train':  train_data,
    'val': val_data,
    'test': test_data
}

In [ ]:
# print shapes
print("Train Patients: ")
for patient_id, data in all_data['train'].items():
    print(f'Patient ID: {patient_id}; HRV shape: {data["hrv"].shape}; Activity shape: {data["activity"].shape}')

print("---------------")
print("Validation Patients: ")
# print shapes
for patient_id, data in all_data['val'].items():
    print(f'Patient ID: {patient_id}; HRV shape: {data["hrv"].shape}; Activity shape: {data["activity"].shape}')

print("---------------")
print("Test Patients: ")
# print shapes
for patient_id, data in all_data['test'].items():
    print(f'Patient ID: {patient_id}; HRV shape: {data["hrv"].shape}; Activity shape: {data["activity"].shape}')

In [ ]:


def segment_data(data, window_size, step_size):
    segments = []
    for start in range(0, len(data) - window_size + 1, step_size):
        segment = data[start:start + window_size]
        segments.append(segment)
    return segments

def normalize_data(data):
    scaler = RobustScaler()
    scaled_data = scaler.fit_transform(data)
    return scaled_data

def preprocessData_create_windows(config, data):
    window_size = config['window_size']
    step_size = window_size // 2
    processed_data = {}

    for patient_id, patient_data in data.items():
        # Segment and normalize HRV and activity data
        hrv_segments = segment_data(patient_data['hrv']['HRV'], window_size, step_size)
        activity_segments = segment_data(patient_data['activity']['ACTIVITY'], window_size, step_size)
        hrv_normalized = normalize_data(np.array(hrv_segments).reshape(-1, window_size)).reshape(-1, window_size)
        activity_normalized = normalize_data(np.array(activity_segments).reshape(-1, window_size)).reshape(-1, window_size)
        labels_repeated = np.repeat(patient_data['adhd'], len(hrv_normalized))

        processed_data[patient_id] = {'hrv': hrv_normalized, 'activity': activity_normalized, 'labels': labels_repeated}

    return processed_data

def preprocessData_no_windows(config, data):
    processed_data = {}

    for patient_id, patient_data in data.items():
        hrv_normalized = normalize_data(np.array(patient_data['hrv']['HRV']).reshape(-1, 1)).reshape(-1, 1)
        activity_normalized = normalize_data(np.array(patient_data['activity']['ACTIVITY']).reshape(-1, 1)).reshape(-1, 1)
        labels_repeated = np.repeat(patient_data['adhd'], len(hrv_normalized))

        processed_data[patient_id] = {'hrv': hrv_normalized, 'activity': activity_normalized, 'labels': labels_repeated}

    return processed_data

In [ ]:
class PatientDataset(Dataset):
    def __init__(self, patients_data):
        self.patients_data = patients_data
        self.patient_ids = list(patients_data.keys())
        self.data = [(patient_id, idx) for patient_id in self.patient_ids for idx in range(len(patients_data[patient_id]['hrv']))]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        patient_id, data_idx = self.data[idx]
        patient_data = self.patients_data[patient_id]

        hrv_data = torch.tensor(patient_data['hrv'][data_idx], dtype=torch.float32)
        activity_data = torch.tensor(patient_data['activity'][data_idx], dtype=torch.float32)
        label = torch.tensor(patient_data['labels'][data_idx], dtype=torch.float32)

        return hrv_data, activity_data, label

In [ ]:
class MultimodalADHDNet(nn.Module):
    def __init__(self, output_channels, hidden_size, num_classes, use_batch_norm=True, use_dropout=True, dropout_rate=0.5):
        super(MultimodalADHDNet, self).__init__()
        self.use_batch_norm = use_batch_norm
        self.use_dropout = use_dropout

        # HRV data branch
        self.hrv_conv1 = nn.Conv1d(1, output_channels, kernel_size=3, padding=1)
        if self.use_batch_norm:
            self.hrv_bn1 = nn.BatchNorm1d(output_channels)

        self.hrv_lstm = nn.LSTM(output_channels, hidden_size, batch_first=True)

        # Activity data branch
        self.act_conv1 = nn.Conv1d(1, output_channels, kernel_size=3, padding=1)
        if self.use_batch_norm:
            self.act_bn1 = nn.BatchNorm1d(output_channels)

        self.act_lstm = nn.LSTM(output_channels, hidden_size, batch_first=True)

        # Fully connected layer
        self.fc = nn.Linear(2 * hidden_size, num_classes)
        if self.use_dropout:
            self.dropout = nn.Dropout(dropout_rate)

    def forward(self, hrv_data, act_data):
        # Ensure input data is correctly shaped [batch_size, 1, sequence_length]
        hrv_x = hrv_data.unsqueeze(1)  # Add a channel dimension if missing
        act_x = act_data.unsqueeze(1)  # Add a channel dimension if missing

        hrv_x = self.hrv_conv1(hrv_x)
        if self.use_batch_norm:
            hrv_x = self.hrv_bn1(hrv_x)
        hrv_x = F.relu(hrv_x)
        hrv_x, _ = self.hrv_lstm(hrv_x.permute(0, 2, 1))
        hrv_x = hrv_x[:, -1, :]

        act_x = self.act_conv1(act_x)
        if self.use_batch_norm:
            act_x = self.act_bn1(act_x)
        act_x = F.relu(act_x)
        act_x, _ = self.act_lstm(act_x.permute(0, 2, 1))
        act_x = act_x[:, -1, :]

        x = torch.cat((hrv_x, act_x), dim=1)
        if self.use_dropout:
            x = self.dropout(x)
        x = self.fc(x)
        return torch.sigmoid(x)

In [ ]:
configurations = [
    {'config_id': 1, 'optimizer_name': 'adam', 'output_channels': 10, 'window_size': 50, 'batch_size': 64, 'hidden_size': 64, 'num_classes': 1, 'learning_rate': 0.001, 'num_epochs': 100, 'use_dropout': True, 'dropout_rate': 0.5, 'use_batch_norm': True},
    {'config_id': 2, 'optimizer_name': 'adam', 'output_channels': 5, 'window_size': 20, 'batch_size': 32, 'hidden_size': 64, 'num_classes': 1, 'learning_rate': 0.01, 'num_epochs': 80, 'use_dropout': True, 'dropout_rate': 0.3, 'use_batch_norm': True},
    {'config_id': 3, 'optimizer_name': 'adam', 'output_channels': 32, 'window_size': 70, 'batch_size': 32, 'hidden_size': 32, 'num_classes': 1, 'learning_rate': 0.01, 'num_epochs': 150, 'use_dropout': True, 'dropout_rate': 0.1, 'use_batch_norm': True},
    {'config_id': 4, 'optimizer_name': 'adam', 'output_channels': 64, 'window_size': 80, 'batch_size': 32, 'hidden_size': 32, 'num_classes': 1, 'learning_rate': 0.01, 'num_epochs': 150, 'use_dropout': True, 'dropout_rate': 0.1, 'use_batch_norm': True},
    {'config_id': 5, 'optimizer_name': 'adam', 'output_channels': 8, 'window_size': 40, 'batch_size': 128, 'hidden_size': 128, 'num_classes': 1, 'learning_rate': 0.05, 'num_epochs': 120, 'use_dropout': True, 'dropout_rate': 0.2, 'use_batch_norm': True},
    {'config_id': 6, 'optimizer_name': 'sgd', 'output_channels': 3, 'window_size': 10, 'batch_size': 16, 'hidden_size': 32, 'num_classes': 1, 'learning_rate': 0.001, 'num_epochs': 50, 'use_dropout': True, 'dropout_rate': 0.5, 'use_batch_norm': True, 'momentum': 0.9},
    {'config_id': 7, 'optimizer_name': 'sgd', 'output_channels': 32, 'window_size': 70, 'batch_size': 32, 'hidden_size': 32, 'num_classes': 1, 'learning_rate': 0.01, 'num_epochs': 150, 'use_dropout': True, 'dropout_rate': 0.5, 'use_batch_norm': True, 'momentum': 0.3},
    {'config_id': 8, 'optimizer_name': 'sgd', 'output_channels': 64, 'window_size': 80, 'batch_size': 32, 'hidden_size': 32, 'num_classes': 1, 'learning_rate': 0.01, 'num_epochs': 150, 'use_dropout': True, 'dropout_rate': 0.1, 'use_batch_norm': True, 'momentum': 0.2},
    {'config_id': 9, 'optimizer_name': 'adam', 'output_channels': 3, 'window_size': 10, 'batch_size': 16, 'hidden_size': 32, 'num_classes': 1, 'learning_rate': 0.001, 'num_epochs': 50, 'use_dropout': True, 'dropout_rate': 0.5, 'use_batch_norm': True},
    {'config_id': 10, 'optimizer_name': 'adam', 'output_channels': 16, 'window_size': 60, 'batch_size': 32, 'hidden_size': 32, 'num_classes': 1, 'learning_rate': 0.01, 'num_epochs': 80, 'use_dropout': True, 'dropout_rate': 0.1, 'use_batch_norm': True},
    {'config_id': 11, 'optimizer_name': 'adam', 'output_channels': 6, 'window_size': 30, 'batch_size': 64, 'hidden_size': 128, 'num_classes': 1, 'learning_rate': 0.1, 'num_epochs': 120, 'use_dropout': False, 'dropout_rate': 0.0, 'use_batch_norm': True},
    {'config_id': 12, 'optimizer_name': 'sgd', 'output_channels': 16, 'window_size': 60, 'batch_size': 32, 'hidden_size': 32, 'num_classes': 1, 'learning_rate': 0.01, 'num_epochs': 80, 'use_dropout': True, 'dropout_rate': 0.1, 'use_batch_norm': True, 'momentum': 0.4},
    {'config_id': 13, 'optimizer_name': 'sgd', 'output_channels': 5, 'window_size': 20, 'batch_size': 32, 'hidden_size': 64, 'num_classes': 1, 'learning_rate': 0.01, 'num_epochs': 100, 'use_dropout': True, 'dropout_rate': 0.3, 'use_batch_norm': True, 'momentum': 0.8},
    {'config_id': 14, 'optimizer_name': 'sgd', 'output_channels': 8, 'window_size': 40, 'batch_size': 128, 'hidden_size': 128, 'num_classes': 1, 'learning_rate': 0.05, 'num_epochs': 120, 'use_dropout': True, 'dropout_rate': 0.2, 'use_batch_norm': True, 'momentum': 0.6},
    {'config_id': 15, 'optimizer_name': 'sgd', 'output_channels': 6, 'window_size': 30, 'batch_size': 64, 'hidden_size': 128, 'num_classes': 1, 'learning_rate': 0.1, 'num_epochs': 120, 'use_dropout': False, 'dropout_rate': 0.0, 'use_batch_norm': True, 'momentum': 0.7},
    {'config_id': 16, 'optimizer_name': 'sgd', 'output_channels': 10, 'window_size': 50, 'batch_size': 64, 'hidden_size': 64, 'num_classes': 1, 'learning_rate': 0.001, 'num_epochs': 100, 'use_dropout': True, 'dropout_rate': 0.5, 'use_batch_norm': True, 'momentum': 0.5}
]


In [ ]:

def calculate_metrics(outputs, labels):
    predicted = outputs.detach().round()
    accuracy = accuracy_score(labels.cpu().numpy(), predicted.cpu().numpy())
    precision = precision_score(labels.cpu().numpy(), predicted.cpu().numpy(), zero_division=0)
    recall = recall_score(labels.cpu().numpy(), predicted.cpu().numpy(), zero_division=0)
    f1 = f1_score(labels.cpu().numpy(), predicted.cpu().numpy(), zero_division=0)
    return accuracy, precision, recall, f1

def train_epoch(model, data_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    total_accuracy, total_precision, total_recall, total_f1 = 0, 0, 0, 0
    total_batches = 0

    for hrv_data, activity_data, labels in data_loader:
        hrv_data, activity_data, labels = hrv_data.to(device), activity_data.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(hrv_data, activity_data).squeeze(1)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * labels.size(0)
        accuracy, precision, recall, f1 = calculate_metrics(outputs, labels)
        total_accuracy += accuracy
        total_precision += precision
        total_recall += recall
        total_f1 += f1
        total_batches += 1

    average_loss = running_loss / len(data_loader.dataset)
    average_accuracy = total_accuracy / total_batches
    average_precision = total_precision / total_batches
    average_recall = total_recall / total_batches
    average_f1 = total_f1 / total_batches

    return average_loss, average_accuracy, average_precision, average_recall, average_f1

def validate_epoch(model, data_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    total_accuracy, total_precision, total_recall, total_f1 = 0, 0, 0, 0
    total_batches = 0

    with torch.no_grad():
        for hrv_data, activity_data, labels in data_loader:
            hrv_data, activity_data, labels = hrv_data.to(device), activity_data.to(device), labels.to(device)
            outputs = model(hrv_data, activity_data).squeeze(1)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * labels.size(0)
            accuracy, precision, recall, f1 = calculate_metrics(outputs, labels)
            total_accuracy += accuracy
            total_precision += precision
            total_recall += recall
            total_f1 += f1
            total_batches += 1

    average_loss = running_loss / len(data_loader.dataset)
    average_accuracy = total_accuracy / total_batches
    average_precision = total_precision / total_batches
    average_recall = total_recall / total_batches
    average_f1 = total_f1 / total_batches

    return average_loss, average_accuracy, average_precision, average_recall, average_f1


def test_model(model, data_loader, device):
    model.eval()
    all_outputs = []
    all_labels = []

    with torch.no_grad():
        for hrv_data, activity_data, labels in data_loader:
            hrv_data, activity_data, labels = hrv_data.to(device), activity_data.to(device), labels.to(device)
            outputs = model(hrv_data, activity_data)
            all_outputs.append(outputs)
            all_labels.append(labels)

    all_outputs = torch.cat(all_outputs).squeeze(1)
    all_labels = torch.cat(all_labels)

    accuracy, precision, recall, f1 = calculate_metrics(all_outputs, all_labels)

    return accuracy, precision, recall, f1



In [ ]:
def run_experiment(config, all_data):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = MultimodalADHDNet(
        output_channels=config['output_channels'],
        hidden_size=config['hidden_size'],
        num_classes=config['num_classes'],
        use_batch_norm=config['use_batch_norm'],
        use_dropout=config['use_dropout'],
        dropout_rate=0.5
    ).to(device)

    # Initialize the optimizer based on the configuration's optimizer name
    if config['optimizer_name'] == 'adam':
        optimizer = torch.optim.Adam(model.parameters(), lr=config['learning_rate'])
    elif config['optimizer_name'] == 'sgd':
        optimizer = torch.optim.SGD(model.parameters(), lr=config['learning_rate'], momentum=config['momentum'])

    criterion = nn.BCELoss()

    train_data = preprocessData_no_windows(config, all_data['train'])
    val_data = preprocessData_no_windows(config, all_data['val'])
    test_data = preprocessData_no_windows(config, all_data['test'])

    train_loader = DataLoader(PatientDataset(train_data), batch_size=config['batch_size'], shuffle=True)
    val_loader = DataLoader(PatientDataset(val_data), batch_size=config['batch_size'], shuffle=False)
    test_loader = DataLoader(PatientDataset(test_data), batch_size=config['batch_size'], shuffle=False)

    final_train_metrics = {}
    final_val_metrics = {}
    history = {'train_loss': [], 'val_loss': [], 'train_acc': [], 'val_acc': []}

    for epoch in range(config['num_epochs']):
        train_loss, train_acc, train_prec, train_rec, train_f1 = train_epoch(model, train_loader, criterion, optimizer, device)
        val_loss, val_acc, val_prec, val_rec, val_f1 = validate_epoch(model, val_loader, criterion, device)

        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)

        print(f"Epoch {epoch + 1}/{config['num_epochs']}: Train Loss: {round(train_loss, 4)}, Train Acc: {round(train_acc, 4)}, Val Loss: {round(val_loss, 4)}, Val Acc: {round(val_acc, 4)}")

        if epoch == config['num_epochs'] - 1:
            final_train_metrics = {
                'Train Loss': round(train_loss, 2),
                'Train Accuracy': round(train_acc, 2),
                'Train Precision': round(train_prec, 2),
                'Train Recall': round(train_rec, 2),
                'Train F1': round(train_f1, 2)
            }
            final_val_metrics = {
                'Val Loss': round(val_loss, 2),
                'Val Accuracy': round(val_acc, 2),
                'Val Precision': round(val_prec, 2),
                'Val Recall': round(val_rec, 2),
                'Val F1': round(val_f1, 2)
            }

    test_accuracy, test_precision, test_recall, test_f1 = test_model(model, test_loader, device)
    test_metrics = {
        'Test Accuracy': round(test_accuracy, 2),
        'Test Precision': round(test_precision, 2),
        'Test Recall': round(test_recall, 2),
        'Test F1': round(test_f1, 2)
    }

    return history, {**final_train_metrics, **final_val_metrics, **test_metrics}


In [ ]:
results = []
config_histories = {}
for config in configurations:
    print(f"Configuration: {config}")
    history, test_results = run_experiment(config, all_data)
    config_histories[config['config_id']] = history
    results.append({'config': config, **test_results})

In [ ]:
def plot_all_configs(histories):
    for config_id, history in histories.items():
        plt.figure(figsize=(12, 5))
        plt.subplot(1, 2, 1)
        plt.plot(history['train_loss'], label='Train Loss')
        plt.plot(history['val_loss'], label='Validation Loss')
        plt.title(f'Config {config_id} - Loss Over Epochs')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()

        plt.subplot(1, 2, 2)
        plt.plot(history['train_acc'], label='Train Accuracy')
        plt.plot(history['val_acc'], label='Validation Accuracy')
        plt.title(f'Config {config_id} - Accuracy Over Epochs')
        plt.xlabel('Epoch')
        plt.ylabel('Accuracy')
        plt.legend()

        plt.show()

# Plotting all configurations after running all experiments
plot_all_configs(config_histories)

In [ ]:
# Convert results to DataFrame and save to CSV
results_df = pd.DataFrame(results)
results_df.to_csv('results_batch05_best_tunned_no_windows.csv', index=False)